# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

Step 1: Assign each point to a cluster N at random \
Step 2: Calculate the mean position of each cluster using the previous assignments. \
Step 3: Loop through the points - reassign each point to the cluster to whose center it is closest. \
Step 4: Recompute centroids and repeat this process until the centers stop moving around.

#### b) Create the builder function for step 1.

In [69]:
label_randomly <- function(n_points, n_clusters){
  sample(((1:n_points) %% n_clusters)+1, n_points, replace=F)
}

#### c) Create the builder function for step 2.

In [70]:
get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [71]:
assign_cluster_fast <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

#### e) Create the builder function for step 4.

In [72]:
kmeans_done <- function(old_means, new_means, eps=1e-6){
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if(m < eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [73]:
mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it){
    labels <- assign_cluster_fast(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if(kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
      it <- it + 1
      if(verbose){
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [74]:
#Question 2

# a)
library(tidyverse)
setwd('C:/Users/keert/BIOS512/R_datasets')
voltages_df <- read_csv('voltages_df.csv', show_col_types = FALSE)

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [76]:
# b)
results <- mykmeans(voltages_df, n_clusters = 3)

results$labels
results$means

[1] 2 3 3 3 3 3 1 1 3 2 2 2 1 2 3 2 2 2 2 2 3 3 3 3 3 2 2 2 1 1 1 1 3 1 1 3 3
 [38] 2 1 3 2 2 2 1 3 1 1 3 1 1 3 1 2 2 1 2 3 2 1 3 2 2 3 2 1 1 3 1 2 1 2 3 1 1
 [75] 3 3 2 1 1 1 1 1 3 1 1 3 3 3 2 1 1 1 3 1 3 3 1 3 3 1 1 1 3 1 2 2 1 2 2 3 2
[112] 2 3 1 2 1 2 2 3 3 3 3 2 2 1 3 3 1 2 1 3 2 3 1 2 3 3 3 2 1 2 3 3 1 2 2 3 2
[149] 2 1 2 1 3 3 1 2 1 1 3 2 1 1 2 3 2 1 3 2 3 2 2 1 3 2 1 1 3 1 2 3 1 3 1 2 1
[186] 3 3 2 3 2 2 3 2 2 1 1 2 3 2 2 1 2 1 1 3 2 3 1 3 2 3 1 2 1 1 3 1 1 3 3 3 3
[223] 3 3 3 1 1 1 1 2 3 2 1 3 3 1 1 2 2 1 1 2 1 1 3 1 2 1 1 3 1 3 2 1 3 1 1 2 2
[260] 2 3 3 1 2 3 3 3 1 1 1 2 3 3 3 1 2 1 1 3 3 1 2 1 2 2 3 1 3 3 1 1 2 1 1 2 3
[297] 2 1 1 3 3 1 2 2 2 1 2 3 3 3 2 2 2 2 2 2 3 2 3 2 2 1 3 1 2 2 2 2 3 2 3 2 1
[334] 2 3 3 1 1 2 3 1 3 1 1 2 1 3 1 2 2 1 3 3 3 2 1 3 2 1 1 3 3 3 3 2 1 2 3 2 2
[371] 2 2 2 3 3 1 1 2 2 2 2 1 1 2 2 1 1 3 3 1 2 3 3 3 3 1 3 3 1 2 2 2 2 3 3 2 2
[408] 1 1 1 3 1 1 2 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 3 1 1 3 1 2 3 2 2 2 1 1 1 1 2
[445] 3 2 2 2 1 3 2 2 2 3 3 2 3 3 2 1 3 3 2 1 3 3 3 2 1 1 2 1 2 3 3 1 2 1 2 3 3
[482] 1 2 1 1 1 3 1 1 1 1 1 3 2 2 1 1 2 3 3 3 2 2 2 3 3 1 1 2 1 2 3 3 1 1 2 1 1
[519] 3 2 2 2 3 1 2 3 1 1 2 3 1 1 1 1 2 2 3 2 2 3 3 3 3 1 2 2 1 2 2 2 3 3 2 3 1
[556] 1 3 1 1 1 3 1 1 3 1 1 3 1 1 2 1 3 1 3 1 3 3 3 2 1 1 1 1 2 3 1 1 1 3 1 1 3
[593] 3 3 1 2 3 3 2 3 1 3 3 1 2 2 1 1 1 3 1 2 3 2 1 3 2 3 3 2 1 3 2 1 1 2 3 2 1
[630] 3 2 1 2 3 3 2 3 2 2 2 2 2 3 3 2 1 2 1 1 2 1 3 1 2 1 2 2 2 3 2 2 3 3 2 2 3
[667] 2 3 1 1 3 2 3 2 2 1 3 3 2 1 2 3 1 1 2 2 3 2 3 3 1 1 1 1 3 3 3 2 1 3 2 1 1
[704] 2 2 1 3 1 1 2 3 2 1 3 2 3 3 2 3 3 2 2 3 3 2 3 1 2 2 2 2 2 3 3 2 2 3 3 2 3
[741] 2 3 1 2 2 2 3 2 1 3 3 2 2 1 2 2 1 2 1 1 2 2 3 1 2 3 1 3 3 2 1 3 1 2 3 2 2
[778] 1 2 1 3 2 2 3 1 3 2 3 2 1 2 2 3 3 3 2 3 2 1 3 3 3 3 2 2 1 3 2 2 2 2 1 1 2
[815] 1 2 3 2 3 1 3 3 3 3 3 1 1 3 1 1 1 1 1 1 3 2 2 3 1 1 2 3 1 1 1 1 2 2 3 1 3
[852] 2 1 1 1 1 2 2 1 1 1 1 1 2 3 1 3 2 3 3 1 2 1 1 2 2 3 2 2 2 1 3 1 3 3 2 2 1
[889] 1 3 2 1 1 3 3 3 3 2 1 1

label__,0,1.00401606425703,2.00803212851406,3.01204819277108,4.01606425702811,5.02008032128514,6.02409638554217,7.0281124497992,8.03212851405623,⋯,240.963855421687,241.967871485944,242.971887550201,243.975903614458,244.979919678715,245.983935742972,246.987951807229,247.991967871486,248.995983935743,250
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,⋯,0.3364266,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245
2,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,⋯,-0.7900387,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393
3,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,⋯,-0.9107472,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784


#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [77]:
# c)
voltages_matrix <- as.matrix(voltages_df)
results <- kmeans(voltages_matrix, centers = 3)

results$cluster
results$centers  

[1] 3 1 1 1 1 1 2 2 1 3 3 3 2 3 1 3 3 3 3 3 1 1 1 1 1 3 3 3 2 2 2 2 1 2 2 1 1
 [38] 3 2 1 3 3 3 2 1 2 2 1 2 2 1 2 3 3 2 3 1 3 2 1 3 3 1 3 2 2 1 2 3 2 3 1 2 2
 [75] 1 1 3 2 2 2 2 2 1 2 2 1 1 1 3 2 2 2 1 2 1 1 2 1 1 2 2 2 1 2 3 3 2 3 3 1 3
[112] 3 1 2 3 2 3 3 1 1 1 1 3 3 2 1 1 2 3 2 1 3 1 2 3 1 1 1 3 2 3 1 1 2 3 3 1 3
[149] 3 2 3 2 1 1 2 3 2 2 1 3 2 2 3 1 3 2 1 3 1 3 3 2 1 3 2 2 1 2 3 1 2 1 2 3 2
[186] 1 1 3 1 3 3 1 3 3 2 2 3 1 3 3 2 3 2 2 1 3 1 2 1 3 1 2 3 2 2 1 2 2 1 1 1 1
[223] 1 1 1 2 2 2 2 3 1 3 2 1 1 2 2 3 3 2 2 3 2 2 1 2 3 2 2 1 2 1 3 2 1 2 2 3 3
[260] 3 1 1 2 3 1 1 1 2 2 2 3 1 1 1 2 3 2 2 1 1 2 3 2 3 3 1 2 1 1 2 2 3 2 2 3 1
[297] 3 2 2 1 1 2 3 3 3 2 3 1 1 1 3 3 3 3 3 3 1 3 1 3 3 2 1 2 3 3 3 3 1 3 1 3 2
[334] 3 1 1 2 2 3 1 2 1 2 2 3 2 1 2 3 3 2 1 1 1 3 2 1 3 2 2 1 1 1 1 3 2 3 1 3 3
[371] 3 3 3 1 1 2 2 3 3 3 3 2 2 3 3 2 2 1 1 2 3 1 1 1 1 2 1 1 2 3 3 3 3 1 1 3 3
[408] 2 2 2 1 2 2 3 1 1 1 2 1 1 1 3 3 1 3 2 1 1 1 1 2 2 1 2 3 1 3 3 3 2 2 2 2 3
[445] 1 3 3 3 2 1 3 3 3 1 1 3 1 1 3 2 1 1 3 2 1 1 1 3 2 2 3 2 3 1 1 2 3 2 3 1 1
[482] 2 3 2 2 2 1 2 2 2 2 2 1 3 3 2 2 3 1 1 1 3 3 3 1 1 2 2 3 2 3 1 1 2 2 3 2 2
[519] 1 3 3 3 1 2 3 1 2 2 3 1 2 2 2 2 3 3 1 3 3 1 1 1 1 2 3 3 2 3 3 3 1 1 3 1 2
[556] 2 1 2 2 2 1 2 2 1 2 2 1 2 2 3 2 1 2 1 2 1 1 1 3 2 2 2 2 3 1 2 2 2 1 2 2 1
[593] 1 1 2 3 1 1 3 1 2 1 1 2 3 3 2 2 2 1 2 3 1 3 2 1 3 1 1 3 2 1 3 2 2 3 1 3 2
[630] 1 3 2 3 1 1 3 1 3 3 3 3 3 1 1 3 2 3 2 2 3 2 1 2 3 2 3 3 3 1 3 3 1 1 3 3 1
[667] 3 1 2 2 1 3 1 3 3 2 1 1 3 2 3 1 2 2 3 3 1 3 1 1 2 2 2 2 1 1 1 3 2 1 3 2 2
[704] 3 3 2 1 2 2 3 1 3 2 1 3 1 1 3 1 1 3 3 1 1 3 1 2 3 3 3 3 3 1 1 3 3 1 1 3 1
[741] 3 1 2 3 3 3 1 3 2 1 1 3 3 2 3 3 2 3 2 2 3 3 1 2 3 1 2 1 1 3 2 1 2 3 1 3 3
[778] 2 3 2 1 3 3 1 2 1 3 1 3 2 3 3 1 1 1 3 1 3 2 1 1 1 1 3 3 2 1 3 3 3 3 2 2 3
[815] 2 3 1 3 1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 2 1 3 3 1 2 2 3 1 2 2 2 2 3 3 1 2 1
[852] 3 2 2 2 2 3 3 2 2 2 2 2 3 1 2 1 3 1 1 2 3 2 2 3 3 1 3 3 3 2 1 2 1 1 3 3 2
[889] 2 1 3 2 2 1 1 1 1 3 2 2

,0,1.00401606425703,2.00803212851406,3.01204819277108,4.01606425702811,5.02008032128514,6.02409638554217,7.0281124497992,8.03212851405623,9.03614457831325,⋯,240.963855421687,241.967871485944,242.971887550201,243.975903614458,244.979919678715,245.983935742972,246.987951807229,247.991967871486,248.995983935743,250
1,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,-1.0022396,⋯,-0.9107472,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784
2,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,-0.9237437,⋯,0.3364266,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245
3,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,-0.5661321,⋯,-0.7900387,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393


#### d) Are your labels/clusters the same? If not, why? Are your means the same?

The labels and clusters are not the same because cluster numbering is arbitrary. The cluster centers or the means are the same however and therefore produce the same outcome showing that the function works

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

A for loop goes over each observation in the dataset, computes its distance to the cluster centroid, and picks the smallest distance therefore assigning that point to the closest centroid. It is one way to assign clusters for kmeans

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

The other option to assign clusters is to vectorize the code which computes all distances at once using matrix operations. This method is faster. The vectorized process expands the data and means into matrices so that every point and mean pair is aligned, computes all distances at once, and finds the nearest mean for each point by grouping.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectorizing is the more efficient process because it is faster - it computes everything at once rather than computing one by one for each point, which is what the for loop does. The vectorized method is also better for larger datasets

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

kmeans fails for non-spherical clusters or clusters of elongated or irregular shapes. This is due to the assumption of roughly spherical clusters with uniform distribution of size and shape.

## Question 5
#### What assumption do Guassian mixture models make?

Gaussian mixture models assume that the data is drawn from N Gaussian distributions whose individual parameters are estimated from the data. This can handle clusters of different sizes and shapes more easily.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering makes the assumption that two points are more likely to be in the same cluster if they are close to one another. It uses a similarity metric between points rather than distance allowing spectral clustering to capture complex, non-spherical structures that K-means cannot.

## Question 7
#### Define the gap statistic method. What do we use it for?

The gap statistic method is one way to decide the number of clusters we want. It allows us to compare the clustering result for each value of K to what we’d expect if there were no inherent cluster structure. We can compute the dispersion of two clusterings and look at the difference and then find the ideal number of clusters. 